In [1]:
import pandas as pd
import numpy  as np
import datetime

import tensorflow as tf
from tensorflow import keras

In [2]:
border_crossing = pd.read_csv('../data/Border_Crossing_Entry_Data.csv')    
border_crossing

,Port Name,State,Port Code,Border,Date,Measure,Value,Location
0,Calexico East,California,2507,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,34447,POINT (-115.48433000000001 32.67524)
1,Van Buren,Maine,108,US-Canada Border,03/01/2019 12:00:00 AM,Rail Containers Full,428,POINT (-67.94271 47.16207)
2,Otay Mesa,California,2506,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,81217,POINT (-117.05333 32.57333)
3,Nogales,Arizona,2604,US-Mexico Border,03/01/2019 12:00:00 AM,Trains,62,POINT (-110.93361 31.340279999999996)
4,Trout River,New York,715,US-Canada Border,03/01/2019 12:00:00 AM,Personal Vehicle Passengers,16377,POINT (-73.44253 44.990010000000005)
...,...,...,...,...,...,...,...,...
346728,Norton,Vermont,211,US-Canada Border,01/01/1996 12:00:00 AM,Personal Vehicle Passengers,9119,POINT (-71.79 45.01)
346729,Columbus,New Mexico,2406,US-Mexico Border,01/01/1996 12:00:00 AM,Personal Vehicles,29486,POINT (-107.63 31.78)
346730,Hansboro,North Dakota,3415,US-Canada Border,01/01/1996 12:00:00 AM,Rail Containers Full,0,POINT (-99.35 49)
346731,Calexico East,California,2507,US-Mexico Border,01/01/1996 12:00:00 AM,Buses,0,POINT (-115.39 32.67)


In [3]:
# Extract time features

# # It turns out time and day is always the same
# def parse_date(s): 
#     [date, time, am_pm]    = s.strip().split()
#     [day, month, year]     = list(map(int, date.split(sep='/')))
#     [hour, minute, second] = list(map(int, time.split(sep=':')))
#     if am_pm == 'AM':
#         if hour == 12:
#             hour = 0
#     else:
#         if hour < 12:
#             hour +=12
#     return datetime.datetime(year=year, month=month, day=day, hour=hour, minute=minute, second=second)

def parse_date(s): 
    date, *_ = s.strip().split()
    [month, day, year] = list(map(int, date.split(sep='/')))
    return datetime.date(year, month, day)
    
border_crossing['Date'] = border_crossing['Date'].apply(parse_date)
border_crossing = border_crossing.sort_values(by='Date')
border_crossing

,Port Name,State,Port Code,Border,Date,Measure,Value,Location
346732,Presidio,Texas,2403,US-Mexico Border,1996-01-01,Trucks,347,POINT (-104.39000000000001 29.56)
345903,Whitlash,Montana,3321,US-Canada Border,1996-01-01,Truck Containers Full,0,POINT (-111.26000000000002 49)
345904,Jackman,Maine,104,US-Canada Border,1996-01-01,Truck Containers Full,2103,POINT (-70.4 45.81)
345905,Hidalgo,Texas,2305,US-Mexico Border,1996-01-01,Truck Containers Full,9794,POINT (-98.27 26.1)
345906,Boundary,Washington,3015,US-Canada Border,1996-01-01,Pedestrians,0,POINT (-117.62999999999998 49)
...,...,...,...,...,...,...,...,...
527,Calexico,California,2503,US-Mexico Border,2019-03-01,Personal Vehicles,413457,POINT (-115.49806000000001 32.67889)
526,Richford,Vermont,203,US-Canada Border,2019-03-01,Train Passengers,30,POINT (-72.67832000000001 44.98588)
525,Eastport,Idaho,3302,US-Canada Border,2019-03-01,Personal Vehicle Passengers,9719,POINT (-116.18027999999998 48.99944)
523,Laurier,Washington,3016,US-Canada Border,2019-03-01,Rail Containers Full,196,POINT (-118.22302 48.99892)


In [ ]:
border_crossing['Year'] = border_crossing['Date'].apply(lambda d: d.year - 1996)
border_crossing['Month'] = border_crossing['Date'].apply(lambda d: d.month - 1)

for idx, row in border_crossing.iterrows():
    row['Total Month'] = row['Year'] * 12 + row['Month']

border_crossing.head()

In [ ]:
del border_crossing['Location']

for col in ['Port Name', 'State', 'Port Code', 'Border', 'Measure']:
    border_crossing[col] = border_crossing[col].astype('category')
    dummies = pd.get_dummies(border_crossing[col],prefix=col)
    border_crossing[dummies.columns] = dummies
    del border_crossing[col]

border_crossing